In [1]:
# Identifica as subpastas da pasta mãe

from __future__ import print_function
import pickle
import os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage
from apiclient.http import MediaFileUpload, MediaIoBaseDownload
import io
from apiclient import errors
from apiclient import http
import logging
from google.auth.transport.requests import Request
from getfilelistpy import getfilelist
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import os

from apiclient import discovery


CLIENT_SECRET = "/home/bruna_ramos/Downloads/client_secrets.json"

# If modifying these scopes, delete the file token.pickle.
#SCOPES = ['https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/drive.metadata.readonly',\
          #'https://www.googleapis.com/auth/drive.readonly','https://www.googleapis.com/auth/drive.file']
SCOPES=[
    'https://www.googleapis.com/auth/drive']

creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET, SCOPES)  # credentials.json download from drive API
        creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
service = build('drive', 'v3', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=887183825426-60s9l25rm89kp2ui22b1asrs58am4vgv.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=XEK377mkgDh5hlVo9c8sPRoxHnPrQd&access_type=offline


In [ ]:


#Com a pasta raíz informada identifica a pasta adicionada mais recentemente (batch atual) e traz as subpastas
def folders_to_current_batch(FOLDER_ID):

#Folder_id = '1M4rfUSYNxbK4qE-of3e4hGcnm4h__ITP' # Enter The Downloadable folder ID From Shared Link
    folder_id=FOLDER_ID

#retorna a pasta do batch atual
    results_folder_1 = service.files().list( q="mimeType='application/vnd.google-apps.folder' and parents in '"+\
                                            folder_id+"' and trashed = false",fields=\
                                            "nextPageToken, files(id, name)",pageSize=400).execute()
    items_folder_1 = results_folder_1.get('files')

    for items in items_folder_1:
        if(items['name']=='sentenca'):
            folder_2=items['id']
            

    results_folder_2 = service.files().list(q=\
                                            "mimeType='application/vnd.google-apps.folder' and parents in '"+\
                                            folder_2+"' and trashed = false",fields=\
                                            "nextPageToken, files(id, name)",pageSize=400).execute()
    items_folder_2 = results_folder_2.get('files', [])
    
    for items in items_folder_2:
        folder_3=items['id']
    
    results_folder_3 = service.files().list(q="mimeType='application/vnd.google-apps.folder' and parents in '"+\
                                            folder_3+"' and trashed = false",fields=\
                                            "nextPageToken, files(id, name, createdTime)",\
                                            orderBy="createdTime desc").execute()
    items_folder_3 = results_folder_3.get('files', [])    
   

    batch_atual=items_folder_3[0]['id']
    name_batch_atual=items_folder_3[0]['name']

    results_batch = service.files().list(q="mimeType='application/vnd.google-apps.folder' and parents in '"+\
                                         batch_atual+"' and trashed = false",fields=\
                                         "nextPageToken, files(id, name)",pageSize=400).execute()

    dic = results_batch.get('files', [])

    if not dic:
        print('No files found.')
    else:
        #print('Files:')
        total=[]
        for i in range(len(dic)):
            pastas=[]
            for key,value in dic[i].items():
                pastas.append((key,value,name_batch_atual))
            total.append(pastas)
    return total


pastas=folders_to_current_batch('17XL23HvyVeuvzKvPFPRWZNu_w6SUMUgX')

def check_folder_to_annotator(folder_id,diretorio):

    
#Identifica os anotadores
    scope = ['https://spreadsheets.google.com/feeds']

    credentials = ServiceAccountCredentials.from_json_keyfile_name(diretorio+'py-sheets-e691f630640c.json', scope)

    gc = gspread.authorize(credentials)

    #folder_id='17XL23HvyVeuvzKvPFPRWZNu_w6SUMUgX'

    resource = {
        "oauth2": creds,
        "fields": "files(name,id)",
        "id":folder_id,
                }
    res = getfilelist.GetFileList(resource) 
    dic = res.get('fileList')


    if not dic:
        print('No files found.')
    else:
        for i in dic:
#Filtra para capturar apenas a planilha 'arquivos' que contém o registro dos arquivos trabalhados
            for arquivo in i['files']:
                if 'annotators_data' in arquivo['name']:
                    lista_anotadores=arquivo


    ident=lista_anotadores['id']
    
    wks = gc.open_by_key(ident).sheet1
    data = wks.get_all_values()
    headers = data.pop(0)
    df = pd.DataFrame(data, columns=headers)
    
#Verfifica se todos os anotadores possuem pastas  
    lista_anotadores=[]
    for anotador in df['anotadores']:
        lista_anotadores.append(anotador)
    
#pastas=identifica_pastas_do_batch_atual('1M4rfUSYNxbK4qE-of3e4hGcnm4h__ITP')
    
    lista_pastas=[]
    for pasta in pastas:
        lista_pastas.append((pasta[1][1]))

    lista_final = list(set(lista_anotadores) - set(lista_pastas ))

    if  not lista_final:
        print("\n Todos anotadores possuem pastas com seus nomes!")
    else:
        print("\n Nem todos anotadores possuem pastas, o diretório em falta é:",lista_final)


check_folder_to_annotator('17XL23HvyVeuvzKvPFPRWZNu_w6SUMUgX',"/home/bruna_ramos/Downloads/")

def find_files(folder_id,f_name):
    folder_name = f_name
    resource = {
        "oauth2": creds,
        "fields": "files(name,id)",
        "id":folder_id,
                }
    res = getfilelist.GetFileList(resource)  

#Recebe todos os arquivos abaixo do diretório informado
    dic = res.get('fileList')
    arquivos=[]
    if not dic:
        print('No files found.')
    else:
        for i in dic:
#Filtra para capturar apenas a planilha 'arquivos' que contém o registro dos arquivos trabalhados
            for arquivo in i['files']:
                if 'arquivos' in arquivo['name']:
                    arquivos.append(arquivo['name'])
    print("\n Lista de arquivos",folder_name,arquivos)


#Gera uma lista com todos os arquivos identificados como arquivos de controle para cada anotador
lista=[]
for i in range(len(pastas)):
    lista.append(find_files(pastas[i][0][1],pastas[i][1][1]))

    
# Verifica se há todas as  pastas (txt,xml e pdf)para cada anotador       

def check_folders_and_files(folder_id,folder_name,folder_txt,folder_pdf,folder_xml):

    folder_name=folder_name
#busca todas as subpastas da pasta indicada

    results=service.files().list(q="mimeType='application/vnd.google-apps.folder' and parents in '"+folder_id+"' and trashed = false",fields="nextPageToken, files(id, name)",pageSize=400).execute()

    dic = results.get('files', [])


    list_folders=[]

    diretorio_ok=[]
    diretorio_nok=[]
    if not dic:
        print('No folders found.')
    else:
        for i in dic:
            list_folders.append(i['name'])
            #list_folders_id_name.append((i['name'],i['id']))
    if (folder_txt in list_folders) and (folder_xml in list_folders)\
    and (folder_pdf in list_folders):
         diretorio_ok.append((i['name'],i['id']))
    if (folder_txt  not in list_folders):
        print("\n A pasta 'arquivos_texto' não está disponível para:",folder_name)
        
    if ('arquivos_resultado' not in list_folders):
        print("\n A pasta 'arquivos_resultado' não está disponível para:",folder_name)
    if ('arquivos_originais' not in list_folders):
        print("\n A pasta 'arquivos_originais' não está criada para:",folder_name)
           

#percorre todas os arquivos das pastas com os arquivos para verficar se há algum arquivo fora do formato esperado            
    inconsistentes=[]
    for i in dic:
        f_name=i['name']
        f_id=i['id']
        
        resource = {"oauth2": creds,"fields": "files(name,id)", "id":f_id,"name":f_name}
        res = getfilelist.GetFileList(resource)

        dic = res.get('fileList')
        arquivos=[]
        arquivos_inconsistentes=[]
        
        if not dic:
            print('No files found.')
        else:
            for i in dic:
                if not i['files']:
                    print("\n Diretório","'",resource['name'],"'","vazio para",folder_name)
                else:
                    for x in i['files']:
                    
                        arquivos.append(x['name'])

                    if resource['name']==folder_txt:
                        for arquivo in arquivos:
                            if(arquivo[-4::]!='.txt') | ("(" in arquivo):
                                arquivos_inconsistentes.append((folder_name,resource['name'],arquivo))
                    if resource['name']==folder_pdf:
                        for arquivo in arquivos:
                            if(arquivo[-4::] !='.pdf' )| ("(" in arquivo):
                                arquivos_inconsistentes.append((folder_name,resource['name'],arquivo))
                    if f_name==folder_xml:
                        for arquivo in arquivos:
                            if(arquivo[-4::] !='.xml')| ("(" in arquivo):
                                arquivos_inconsistentes.append((resource['name'],arquivo,))
                inconsistentes.append(arquivos_inconsistentes)

    if len(i['files'])!= 0 and len(inconsistentes)!= 0:
            print("\n Há inconsistencia de arquivos no seguinte diretorio:",inconsistentes)
    else:
        pass

lista_pastas_criadas=[]
for i in range(len(pastas)):
    lista_pastas_criadas.append(check_folders_and_files(pastas[i][0][1],pastas[i][1][1],\
                                'arquivos_texto',\
                                'arquivos_originais',\
                                'arquivos_resultado'))
lista_pastas_criadas

              


In [ ]:
#Cria diretórios locais para disponibilizar o arquivo excel de cada anotador
def create_diretory(f_id,f_name,diretorio):
    
    diretorio=diretorio
    
    if not os.path.exists(diretorio+f_name):
        os.mkdir(diretorio+f_name)

for pasta in pastas:
    create_diretory(pasta[0][1],pasta[1][1],"/home/bruna_ramos/Downloads/anotadores/")

In [ ]:
#Download de arquivo xlsx do drive
def downloader(file_id,f_name,service):
    
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(f_name, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print((status.progress() * 100))
        

def download_files_xlsx(Folder_id,f_name,service,diretorio,nome_arquivo):
    
   
    name_folder=f_name
    folder_id=Folder_id
    #os.chdir(diretorio+name_folder)
    
    #results = service.files().list(q="mimeType='application/vnd.google-apps.spreadsheet' and parents in '"+folder_id+"' and trashed = false",fields="nextPageToken, files(id, name)").execute()
    results = service.files().list(q="mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' and parents in '"+folder_id+"' and trashed = false",fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])
    for i in items:
        if nome_arquivo in i['name']:
            try:
                downloader(i['id'],diretorio+name_folder+'/'+i['name'],service)
            except IOError as e:
                print ("I/O error({0}): {1}".format(e.errno, e.strerror))

for pasta in pastas:
    download_files_xlsx(pasta[0][1],pasta[1][1],service,"/home/bruna_ramos/Downloads/anotadores/",'arquivo')


In [ ]:
from datetime import datetime
import re
def concat_files(pastas,diretorio,arquivo):

    appended_data = []
    for pasta in pastas:
        #os.chdir(diretorio+pasta[1][1])
        #if 'xlsx' in arquivo:
        df=pd.read_excel(diretorio+pasta[1][1]+'/'+arquivo)
        df.fillna('',inplace=True)
        appended_data.append(df)

    appended_data = pd.concat(appended_data)
    appended_data.reset_index(inplace=True)
    appended_data['batch_id']= int(pasta[0][2][-3::])
    appended_data['xml_file']='False'
 
    regex="[0-9]{7}[-][0-9]{2}[\.][0-9]{4}[\.][0-9]{1}[\.][0-9]{2}[\.][0-9]{4}"
    appended_data['cnj']=appended_data.to.apply(lambda x:re.findall(regex,x)[0])
    
    appended_data['batch_closing_date']=datetime.now()
    
    appended_data = appended_data[['annotator','batch_closing_date','batch_id','cnj','date','id','prediction'\
                                  ,'to','xml_file']]
    appended_data.fillna("",inplace=True)
    return appended_data
#appended_data.to_excel('appended.xlsx')


df=concat_files(pastas,"/home/bruna_ramos/Downloads/anotadores/","arquivos.xlsx")


In [ ]:
df


In [ ]:
def gera_lista_arquivos_ja_trabalhados(line):
    
    lista_base=['sentenca_homologatoria','sentenca_execucao','sentenca','ata_julgamento']
    lista=[]
    if line.prediction in lista_base:
        lista=line.id
 
    return lista
#",".join(flags)
lista=df.apply(gera_lista_arquivos_ja_trabalhados,axis=1)

lista_arquivos_trabalhados=[l for l in lista if l]
lista_arquivos_trabalhados

In [ ]:
def download_annotation_history(Folder_id,service,diretorio):
    
    folder_id=Folder_id
    #os.chdir(diretorio)
    if not os.path.exists(diretorio+'annotation_history'):
        os.mkdir(diretorio+'annotation_history')

    #navega pelos diretórios para chegar até diretório onde ha o arquivo com o histórico de anotações
    results_folder_1 = service.files().list( q="mimeType='application/vnd.google-apps.folder' and parents in '"+\
                                            folder_id+"' and trashed = false",fields=\
                                            "nextPageToken, files(id, name)",pageSize=400).execute()
    items_folder_1 = results_folder_1.get('files')

    for items in items_folder_1:
        if(items['name']=='sentenca'):
            folder_2=items['id']
            
    results = service.files().list(q="mimeType='text/csv' and parents in '"+folder_2+"' and trashed = false",\
                                   fields="nextPageToken, files(id, name, createdTime)",\
                                            orderBy="createdTime desc").execute()
    items = results.get('files', [])

    try:
        downloader(items[0]['id'],diretorio+'annotation_history/'+items[0]['name'],service)
    except IOError as e:
        print ("I/O error({0}): {1}".format(e.errno, e.strerror))
        

In [ ]:
download_annotation_history('17XL23HvyVeuvzKvPFPRWZNu_w6SUMUgX',service,"/home/bruna_ramos/Downloads/")


In [ ]:
#incluir registros trabalhados na lista de arquivos atualizada

#pega o ultimo arquivo baixado
from pathlib import Path

data_modificacao = lambda f: f.stat().st_mtime

directory = Path("/home/bruna_ramos/Downloads/annotation_history")
files = directory.glob('*.csv')
sorted_files = sorted(files, key=data_modificacao, reverse=True)
arquivo =sorted_files[0]
df_annotation=pd.read_csv(arquivo)
#df_annotation = pd.read_csv(arquivo, parse_dates=['date'])
df_annotation

In [ ]:
def list_files_xml(Folder_id,pasta_mae,pasta_xml):

    folder_id=Folder_id
    folder_name=pasta_mae
    
    results=service.files().list(q="mimeType='application/vnd.google-apps.folder' and parents in '"+folder_id+"' and trashed = false",fields="nextPageToken, files(id, name)",pageSize=400).execute()

    dic = results.get('files', [])
    
    for items in dic:
        if items['name']==pasta_xml:
            folder= items['id']

    results = service.files().list(q="mimeType='text/xml' and parents in '"+folder+"' and trashed = false",fields="nextPageToken, files(id, name)").execute()

    items = results.get('files', [])
    
    list_files_xml=[int(i['name'][0:6]) for i in items]

    arquivos_e_xml=list(set(list_files_xml) ^ set(lista_arquivos_trabalhados))

    print("\n Há inconsistencia entre a anotação da planilha 'arquivo.xlsx' e o(s) documento(s) xml:",arquivos_e_xml," da pasta:", folder_name)


    dicionario_xml = {}

    dicionario_xml[folder_name]=list_files_xml

    return dicionario_xml

lista_xml=[]
for pasta in pastas:
    lista_xml.append((list_files_xml(pasta[0][1],pasta[1][1],'arquivos_resultado')))
lista_xml


In [ ]:
# Define no Dataframe quais documentos listados possuem arquivos xml

import numpy as np
for index,row in df.iterrows(): 
    for i in range(len(lista_xml)):
        for key,value in lista_xml[i].items():
            if row.annotator==key and row.id in value:
                df.at[index, 'xml_file'] = "True"


In [ ]:
# Gera planilha de historicos de anotações

df_trabalhado=df.loc[(df['prediction'] != '')]

batch_atual=int(df.batch_id.unique())

for index,row in df_annotation.iterrows():
    if row.batch_id == batch_atual:
         df_annotation.drop(index, inplace=True)
            


df_annotation=df_annotation.append(df_trabalhado)
df_annotation.fillna("",inplace=True)


#df_annotation.reset_index(inplace=True)
df_annotation.reset_index(drop=True, inplace=True)
df_annotation['date_formated']=df_annotation.date.map(lambda x: str(x)[0:10] if x != "" else x)
df_annotation

#df_anotation.to_csv("/home/bruna_ramos/Downloads/annotation_history_upload/appended.csv")

In [ ]:
def load_annotation_history(diretorio,id_folder):

    data_modificacao = lambda f: f.stat().st_mtime

    directory = Path(diretorio)
    files = directory.glob('*.csv')
    sorted_files = sorted(files, key=data_modificacao, reverse=True)
    arquivo =sorted_files[0]

    now =datetime.now()
    data_e_hora_em_texto = now.strftime('%d%m%Y_%H%M')
    name='annotation_history_'+data_e_hora_em_texto+'.csv'
    file_metadata = {'name': name,'parents':[id_folder]}
    media = MediaFileUpload(arquivo,
                        mimetype='text/csv')
    file = service.files().create(body=file_metadata,
                                    media_body=media,
                                    fields= 'id').execute()
    print ('Arquivo carregado com sucesso!\nFile id:',file.get('id'))
load_annotation_history("/home/bruna_ramos/Downloads/annotation_history_upload",'1aJeV7gPUeazhQv5U3kiWFG4Qzopt5P0s')

In [ ]:
df_annotation= df_annotation[['annotator',
                                'batch_closing_date',
                                'batch_id','cnj',
                                'date','id',
                                'prediction','to','xml_file']]
        
    
df_report=df_annotation

df_report.reset_index(drop=True, inplace=True)
df_report['date_formated']=df_report.date.map(lambda x: str(x)[0:10] if x != "" else x)
   

df_report = df_report.rename(columns={'annotator':'anotador',
                                              'batch_closing_date': 'data_captura_dados',
                                             'batch_id':'batch',
                                             'cnj':'cnj',
                                             'id':'id_documento',
                                             'prediction':'classificacao_arquivo',
                                             'to':'diretorio_s3',
                                              'xml_file':'xml_file',
                                             'date_formated':'data'})

df_report=df_report[['anotador','data_captura_dados','batch','cnj','id_documento','classificacao_arquivo',\
                  'diretorio_s3','xml_file','data']]

df_report.loc[df_report["xml_file"]== False,'xml_file'] = 'sem xml'
df_report.loc[df_report["xml_file"]== True,'xml_file'] = 'xml disponivel'

In [ ]:
analise = df_annotation.groupby(['batch_id','annotator','date_formated','prediction', 'xml_file'])\
['xml_file'].agg(['count']).reset_index()
analise

In [ ]:
done_per_batch=df_report.loc[df_report['classificacao_arquivo']!=''].groupby(['batch'])['batch'].agg(['count']).reset_index()

all_files_per_batch=df_report.loc[df_report['id_documento']].groupby(['batch'])['batch'].agg(['count']).reset_index()
all_files_per_batch
#1 329
#3 842

In [ ]:
df_report

In [ ]:
# define a quantidadde de anotações por dia(de todos os batchs)

batch_atual=1
doc_day_annotator= df_report.loc[(df_report['xml_file']=='xml disponivel') & \
                                 (df_report['classificacao_arquivo']=='sentenca')&(df_report['batch']==batch_atual)].\
 groupby(['data','anotador'])\
['xml_file'].agg(['count']).sort_values(by='data').reset_index()
doc_day_annotator

In [ ]:
# gera grafico de de volume percentual de sentenças e não sentenças de cada batch

all_files= df_report.loc[df_report['classificacao_arquivo']!=''].groupby(['batch'])['batch']\
.agg(['count']).reset_index()
all_files = all_files.set_index(['batch'])

only_sentence=df_report.loc[df_report['classificacao_arquivo'].str.contains('sentenca')].groupby(['batch'])['batch']\
.agg(['count']).reset_index()
only_sentence = only_sentence.set_index(['batch'])

donne_per_batch= all_files.merge(only_sentence, how="outer",on =['batch'])

donne_per_batch = donne_per_batch.rename(columns={'count_x': 'total', 'count_y': 'sentenca'})
donne_per_batch['%_sentenca']=donne_per_batch['sentenca']/donne_per_batch['total']*100
donne_per_batch['%_nao_sentenca']= 100-donne_per_batch['%_sentenca']
donne_per_batch.drop('total', inplace=True, axis=1)
donne_per_batch.drop('sentenca', inplace=True, axis=1)


donne_per_batch.plot(kind='barh', stacked=True,title="Analise do volume de documentos já classificados\n",rot=0)

In [ ]:
donne_per_batch= df_report.loc[df_report['classificacao_arquivo']!=''].groupby(['batch'])['batch']\
.agg(['count']).reset_index()
donne_per_batch = donne_per_batch.set_index(['batch'])
all_files_per_batch=df_report.groupby('batch')['batch'].count()
perc_done=(donne_per_batch['count']/all_files_per_batch)*100

perc_done.plot(kind='barh', stacked=True,title="Volume de documentos já classificados por batch\n")

In [ ]:
all_donne_per_document = df_report.loc[(df_report['classificacao_arquivo']!='')].\
groupby(['classificacao_arquivo'])['id_documento'].agg(['count']).sort_values(by ='count',\
                                                                              ascending=False).reset_index()
all_donne_per_document = all_donne_per_document.set_index(['classificacao_arquivo'])

all_donne_per_document.plot(kind='bar', stacked=True,title="Volume de tipos de documentos (todos os batches)\n",rot=90)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

doc_day_annotator= df_report.loc[(df_report['xml_file']=='xml disponivel') & \
                                 (df_report['classificacao_arquivo'].str.contains('sentenca'))].\
groupby(['data','anotador'])['xml_file'].agg(['count']).sort_values(by='data').reset_index()
datas= df_report.loc[df_report['data']!=''].groupby(['data','anotador'])['data'].agg(['count']).reset_index()
#doc_day_annotator= doc_day_annotator.merge(datas, how="right",on =['anotador']).sort_values(by='data')
print(doc_day_annotator.head(5))
#ax=sns.catplot(x="data",y='count_x',kind='point',data=doc_day_annotator)
#plt.xlabel("data")
#plt.ylabel("contagem")
#plt.title("Sentenças com xml - todos batches-full")
#ax.fig.autofmt_xdate()



In [ ]:
docs_day_annotator= df_report.loc[(df_report['xml_file']=='xml disponivel') & \
                                 (df_report['classificacao_arquivo'].str.contains('sentenca'))].\
groupby(['data','anotador'])['xml_file'].agg(['count']).sort_values(by='data').reset_index()
        
datas=df_report['data'].unique()
anotadores=df_report['anotador'].unique()
index = pd.MultiIndex.from_product([datas, anotadores], names = ["data", "anotador"])

stage_table=pd.DataFrame(index = index).sort_values(by='data').reset_index()
print(stage_table)
print(docs_day_annotator)
        
docs_day_annotator= docs_day_annotator.merge(stage_table, how="outer",on =['data','anotador']).sort_values(by='data')
docs_day_annotator['count'].fillna(0, inplace=True)
print(docs_day_annotator.head(30))

In [ ]:
df.loc[(df.id != '')].groupby(['batch_id'])['batch_id'].agg(['count'])

In [ ]:
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})


with PdfPages("/home/bruna_ramos/Downloads/Charts.pdf") as export_pdf:
    done=pd.DataFrame()

    per_trabalhado= int((df.loc[(df.prediction != '') &(df['batch']==batch_atual)].count()['prediction'])\
                        /(df.count()['id'])*100)

    perc_nao_trabalhado=100-per_trabalhado

    done.set_value(1, 'feito', per_trabalhado)
    done.set_value(1, 'meta', perc_nao_trabalhado)
    done.rename(index={1:"documentos clasificados"},inplace=True)


    done.plot(kind='barh', stacked=True,title="Analise do volume de documentos já classificados\n",rot=0)

    export_pdf.savefig()
    
    plt.close()
    
    donne_per_annotator = df.loc[(df['prediction']!='')&(df['batch']==batch_atual)].groupby(['annotator'])\
    ['id'].agg(['count']).reset_index()


    do_per_annotator = df.loc[(df['id']!='')&(df['batch']==batch_atual)].groupby(['annotator'])\
    ['id'].agg(['count']).reset_index()

    donne_per_annotator= donne_per_annotator.merge(do_per_annotator, left_on='annotator', right_on='annotator')
    donne_per_annotator = donne_per_annotator.set_index(['annotator'])
    donne_per_annotator['feito']= donne_per_annotator['count_x']/donne_per_annotator['count_y']
    donne_per_annotator['meta'] = (donne_per_annotator['count_y'].sub(donne_per_annotator['count_x'], axis=0))/donne_per_annotator['count_y']

    donne_per_annotator.drop('count_x', inplace=True, axis=1)
    donne_per_annotator.drop('count_y', inplace=True, axis=1)

    donne_per_annotator.plot(kind='bar', stacked=True,\
                                 title="Analise do volume de documentos já classificados\n",rot=90)
    
    export_pdf.savefig()    
    plt.close()
    
    donne_per_document = df.loc[(df['prediction']!='')].groupby(['prediction'])\
    ['id'].agg(['count']).sort_values(by ='count',ascending=False).reset_index()
    donne_per_document = donne_per_document.set_index(['prediction'])

    donne_per_document.plot(kind='bar', stacked=True,title="Volume de tipos de documentos\n",rot=90)

    export_pdf.savefig()    
    plt.close()


        
    g=sns.catplot(x="data",y='count',hue='anotador',kind='point',data=doc_day_annotator)
    g._legend.remove()
        #g.set(xlim=(1, 15))
    plt.legend(bbox_to_anchor=(1.0,1.0), title='anotador')
    g.fig.autofmt_xdate()
        #export_pdf.savefig()    
        #plt.close()
        
    
    doc_day= df_report.loc[(df_report['xml_file']=='xml disponivel') & \
                                 (df_report['classificacao_arquivo']=='sentenca')].\
    groupby(['data'])['xml_file'].agg(['count']).sort_values(by='data').reset_index()
        
    ax=sns.catplot(x="data",y='count',kind='point',data=doc_day)
    ax.fig.autofmt_xdate()
    export_pdf.savefig()    
    plt.close()

In [ ]:
df_report.head(2)

In [ ]:

def alertas(row):
    alertas=[]
    lista_todos_arquivos=['acordao',
                        'agravo_instrumento',
                        'alvara_diversos',
                        'alvara_levantamento',
                        'ata_audiencia',
                        'ata_julgamento',
                        'contestacao',
                        'contrarrazoes',
                        'decisao_diversas',
                        'decisao_embargos',
                        'doc_diversos',
                        'embargos_declaratorios',
                        'embargos_execucao',
                        'erro_no_arquivo',
                        'guia_deposito',
                        'guia_diversos',
                        'peticao_inicial',
                        'peticao_intermediaria',
                        'peticao_juntada',
                        'recurso_ordinario',
                        'recurso_revista',
                        'sentenca',
                        'sentenca_homologatoria']
    lista_xml=['sentenca']

    if row.classificacao_arquivo not in lista_xml and row.xml_file=='xml disponivel':
        alertas.append(("inconsistencia na pasta de resultados de: "+row.anotador\
                        +". Não é esperao arquivo xml para o documento do tipo: "\
                        +row.classificacao_arquivo+",id do documento:"+", batch "+(str(row.id_documento))))
    if (row.data == ""):
        alertas.append((" Há data não preenchida para"+row.anotador+", batch "+(str(row.batch))\
                        +"documento"+(str(row.id_documento))))
    if row.classificacao_arquivo not in lista_todos_arquivos:
        alertas.append("Classificação inexistente de "+row.anotador+", batch "+(str(row.batch))+\
                       ", id_documento:"+(str(row.id_documento))+". A classificação '"+ row.classificacao_arquivo+"' é invalida")
        
    return  alertas    
alertas=df_report.apply(alertas,axis=1)

In [ ]:

lista_alertas=[alerta[0] for alerta in alertas if alerta]
print(lista_alertas)

In [ ]:
def escrever_txt(lista_alertas,diretorio):
    with open(diretorio+'log_anotacoes.txt', 'w', encoding='utf-8') as f:
        for nome in lista_alertas:
            f.write(nome + '\n')

def carregar_txt():
    with open('meu_arquivo.txt', 'r', encoding='utf-8') as f:
        return f.readlines()


escrever_txt(lista_alertas,"/home/bruna_ramos/Downloads/")

In [ ]:
def load_drive(diretorio,file_id, nome_do_arquivo, mimetype):

    data_modificacao = lambda f: f.stat().st_mtime

    directory = Path(diretorio)
    files = directory.glob(nome_do_arquivo)
    sorted_files = sorted(files, key=data_modificacao, reverse=True)


    arquivo =sorted_files[0]
    
    media_body = MediaFileUpload(
            arquivo, 
            mimetype=mimetype)

    # Send the request to the API.
    updated_file = service.files().update(
                                fileId=file_id,

                                media_body=media_body).execute()
    print('o seguinte arquivo foi atualizado:',updated_file)
    

load_drive("/home/bruna_ramos/Downloads/",'1KRM-fodrZ6oHpAXKWTCEY1jTPFXZjUoM', 'log_anotacoes.txt', 'text/plain')

In [ ]:
per_day = df_annotation.groupby(['batch_id','date_formated','annotator'])\
['date_formated'].agg(['count']).reset_index()
per_day

In [ ]:
no_xml = df_annotation.loc[(df_annotation['xml_file']=='False') & (df_annotation['prediction']=='sentenca')].groupby(['batch_id','annotator','prediction', 'xml_file'])\
['xml_file'].agg(['count']).reset_index()
no_xml

In [ ]:
com_xml = df_annotation.loc[(df_annotation['xml_file']=='True') & (df_annotation['prediction']=='sentenca')].groupby(['date_formated','prediction', 'xml_file'])\
['xml_file'].agg(['count']).reset_index()
com_xml

In [ ]:
#Dataframe de barras exibindo evolução de documentos classificados
import matplotlib.pyplot as plt
done=pd.DataFrame()

per_trabalhado= int((df[df.prediction != ''].count()['prediction'])/(df.count()['id'])*100)

perc_nao_trabalhado=100-per_trabalhado

done.set_value(1, 'feito', per_trabalhado)
done.set_value(1, 'meta', perc_nao_trabalhado)
done.rename(index={1:"documentos clasificados"},inplace=True)


done.plot(kind='barh', stacked=True,title="Analise do volume de documentos já classificados\n",rot=0)
#plt.savefig("/home/bruna_ramos/Downloads/teste5.pdf")


In [ ]:
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

with PdfPages("/home/bruna_ramos/Downloads/Charts.pdf") as export_pdf:
    done=pd.DataFrame()

    per_trabalhado= int((df[df.prediction != ''].count()['prediction'])/(df.count()['id'])*100)

    perc_nao_trabalhado=100-per_trabalhado

    done.set_value(1, 'feito', per_trabalhado)
    done.set_value(1, 'meta', perc_nao_trabalhado)
    done.rename(index={1:"documentos clasificados"},inplace=True)


    done.plot(kind='barh', stacked=True,title="Analise do volume de documentos já classificados\n",rot=0)

    export_pdf.savefig()
    
    plt.close()
    
    donne_per_annotator = df.loc[(df['prediction']!='')].groupby(['annotator'])\
    ['id'].agg(['count']).reset_index()


    do_per_annotator = df.loc[(df['id']!='')].groupby(['annotator'])\
    ['id'].agg(['count']).reset_index()


    donne_per_annotator= donne_per_annotator.merge(do_per_annotator, left_on='annotator', right_on='annotator')
    donne_per_annotator = donne_per_annotator.set_index(['annotator'])
    donne_per_annotator['feito']= donne_per_annotator['count_x']/donne_per_annotator['count_y']
    donne_per_annotator['meta'] = (donne_per_annotator['count_y'].sub(donne_per_annotator['count_x'], axis=0))/donne_per_annotator['count_y']

    donne_per_annotator.drop('count_x', inplace=True, axis=1)
    donne_per_annotator.drop('count_y', inplace=True, axis=1)

    donne_per_annotator.plot(kind='bar', stacked=True,title="Analise do volume de documentos já classificados\n",rot=0)
    
    export_pdf.savefig()
    
    plt.close()
    
    donne_per_document = df.loc[(df['prediction']!='')].groupby(['prediction'])\
    ['id'].agg(['count']).sort_values(by ='count',ascending=False).reset_index()
    donne_per_document = donne_per_document.set_index(['prediction'])


    donne_per_document.plot(kind='bar', stacked=True,title="Volume de tipos de documentos\n",rot=90)
    export_pdf.savefig()
    
    plt.close()
    
    import seaborn as sns 
    sns.catplot(x="date_formated",y='count',hue='annotator',kind='point',data=anotador_xml)
    export_pdf.savefig()
    
    plt.close()
        

In [ ]:
donne_per_annotator = df.loc[(df['prediction']!='')].groupby(['annotator'])\
['id'].agg(['count']).reset_index()
donne_per_annotator

do_per_annotator = df.loc[(df['id']!='')].groupby(['annotator'])\
['id'].agg(['count']).reset_index()
do_per_annotator

donne_per_annotator= donne_per_annotator.merge(do_per_annotator, left_on='annotator', right_on='annotator')
donne_per_annotator = donne_per_annotator.set_index(['annotator'])
donne_per_annotator['feito']= donne_per_annotator['count_x']/donne_per_annotator['count_y']
donne_per_annotator['meta'] = (donne_per_annotator['count_y'].sub(donne_per_annotator['count_x'], axis=0))/donne_per_annotator['count_y']

donne_per_annotator.drop('count_x', inplace=True, axis=1)
donne_per_annotator.drop('count_y', inplace=True, axis=1)

donne_per_annotator.plot(kind='bar', stacked=True,title="Analise do volume de documentos já classificados\n",rot=0)

In [ ]:
#anotações por tipo de documento

donne_per_document = df.loc[(df['prediction']!='')].groupby(['prediction'])\
['id'].agg(['count']).sort_values(by ='count',ascending=False).reset_index()
donne_per_document = donne_per_document.set_index(['prediction'])


donne_per_document.plot(kind='bar', stacked=True,title="Volume de tipos de documentos\n",rot=90)

In [ ]:
anotador_xml = df_annotation.loc[(df_annotation['xml_file']=='True') & (df_annotation['prediction']=='sentenca')].groupby(['date_formated','annotator', 'xml_file'])\
['xml_file'].agg(['count']).reset_index()
anotador_xml

In [ ]:
#import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

sns.set(style="whitegrid")

f, ax = plt.subplots(figsize=(4.5, 4.5))
sns.despine(f)
sns_plot=sns.lineplot(x="date_formated", y="count",
                hue="annotator",
                data=anotador_xml,ax=ax)
#ax.set_xticklabels(labels, rotation=30)
ticks =sns_plot.set_xticklabels(ax,rotation=45)
#sns_plot.savefig('output.png')



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
ax = sns.catplot(x="date_formated",y='count',hue='annotator',kind='point',data=anotador_xml,legend="down")
ax.fig.autofmt_xdate()
#sns.plt.legend(loc='upper left',bbox_to_anchor=(1,0.5))

In [ ]:
>>> from matplotlib import pyplot as plt
>>> fig = plt.figure()
>>> axis1 = fig.add_subplot(211)
>>> axis1.plot(range(10))
>>> axis2 = fig.add_subplot(212)
>>> axis2.plot(range(10,20))
>>> fig.savefig("multipleplots.png")